In [1]:
import transformers

transformers.logging.set_verbosity_error()

In [2]:
import pandas as pd
from tqdm import tqdm
from filter_paraphrases import filter_paraphrases

In [3]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

## Library version

In [1]:
from russian_paraphrasers import GPTParaphraser

In [2]:
model_name = 'russian-paraphrasers-gpt3'

In [6]:
paraphraser = GPTParaphraser(model_name="gpt3", range_cand=True, make_eval=False)

In [7]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

In [8]:
sentence = "Мерседес это самая быстрая машина на российском рынке."

In [9]:
results = paraphraser.generate(
    sentence, n=10, temperature=1,
    top_k=10, top_p=0.9,
    max_length=100, repetition_penalty=1.5,
    threshold=0.7
)
results

{'origin': 'Мерседес это самая быстрая машина на российском рынке.',
 'results': [{'predictions': ['Merce признали самой быстрой машиной в России.',
    'Самая быстрорастущая компания по производству авто в России.',
    'Самая быстрорастущая марка автомобилей в России.',
    'Merce является самой быстрорастущей маркой автомобилей в России.',
    'Мерседесе назвали самой быстрорастущей маркой в России.',
    'Самая быстрорастущая марка автомобилей в России.',
    'Мерседесе признали самой быстрой машиной в России.',
    'Самая скоростная российская марка авто "мерседесе".',
    'Названа максимальная скорость самой быстрой иномарки в России.',
    'Названа наиболее быстрорастущая марка легковых автомобилей в России.'],
   'best_candidates': ['Самая быстрорастущая компания по производству авто в России.',
    'Мерседесе назвали самой быстрорастущей маркой в России.',
    'Названа наиболее быстрорастущая марка легковых автомобилей в России.',
    'Самая быстрорастущая марка автомобилей в 

In [10]:
results['results']

[{'predictions': ['Merce признали самой быстрой машиной в России.',
   'Самая быстрорастущая компания по производству авто в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Merce является самой быстрорастущей маркой автомобилей в России.',
   'Мерседесе назвали самой быстрорастущей маркой в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Мерседесе признали самой быстрой машиной в России.',
   'Самая скоростная российская марка авто "мерседесе".',
   'Названа максимальная скорость самой быстрой иномарки в России.',
   'Названа наиболее быстрорастущая марка легковых автомобилей в России.'],
  'best_candidates': ['Самая быстрорастущая компания по производству авто в России.',
   'Мерседесе назвали самой быстрорастущей маркой в России.',
   'Названа наиболее быстрорастущая марка легковых автомобилей в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Merce признали самой быстрой машиной в России.',
   'Merce является самой быстро

In [11]:
para_results = []
for i in tqdm(toxic_inputs):
    results = paraphraser.generate(
        i, n=10, temperature=1,
        top_k=10, top_p=0.9,
        max_length=100, repetition_penalty=1.0,
        threshold=0.7
    )
    para_result = []
    for result in results['results']:
        if result['best_candidates']:
            para_result.append(result['best_candidates'][-1])
        else:
            para_result.append(result['predictions'][0])
    para_results.append(' '.join(para_result))

100%|██████████| 800/800 [35:25<00:00,  2.66s/it]


In [12]:
with open(f'data/output/{model_name}-no-repetition-penalty_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence + '\n' for sentence in para_results])

## HuggingFace version

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from russian_paraphrasers.utils import clean

In [5]:
checkpoint = 'alenusch/rugpt3-paraphraser'
model_name = checkpoint.split('/')[1]

In [6]:
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
model.cuda();

In [7]:
sentence = 'а мы просим тебя иди нахуй'
prompt = f'<s>{sentence} === '

In [8]:
encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
encoded_prompt = encoded_prompt.to(model.device)

In [9]:
input_ids = encoded_prompt

In [10]:
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,
    temperature=1,
    top_k=10,
    top_p=0.9,
    repetition_penalty=1.5,
    do_sample=True,
    num_return_sequences=10
)

In [11]:
tokenizer.decode(output_sequences[0])

'<s>а мы просим тебя иди нахуй === ьы должны идти у ииди ты знаешь куда а то нас застукают мама не горюй мэм</s>\n<s>И я хочу их обратно === Это все чего мне хочется</s>\n<s>На Камчатке после землетрясения обнаружили тела трех погибших.   /s>\n<s>Я думаю это будет забавно сэр === Я подумал что было бы здорово устроить'

In [12]:
problematic_outputs = []

In [13]:
def paraphrase(
        text,
        model,
        tokenizer,
        temperature=1,
        top_k=10,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
        stop_token='</s>'
):
    prompt = f'<s>{text} === '
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(model.device)
    max_length = int(encoded_prompt.shape[1] * 1.5 + 10)
    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences
    )
    output_text = tokenizer.decode(output_sequences[0])
    output_text = output_text.replace('=== ===', '===')
    output_text = output_text[: output_text.find(stop_token) if stop_token else None]
    output_text = output_text[len(tokenizer.decode(encoded_prompt[0])):]
    output_text = clean(output_text)
    if not output_text:
        problematic_outputs.append(output_sequences)
    return output_text

In [14]:
paraphrase('а мы просим тебя иди нахуй',
           model,
           tokenizer,
           top_k=20,
           top_p=1.0
           )

'Мы тебя с братом на хуй отправим'

In [25]:
paraphrase_kwargs = {
    'text': 'хуйло ебаное пидорас чтоб ты сдох',
    'model': model,
    'tokenizer': tokenizer,
    'top_k': 20,
    'top_p': 1.0
}

In [29]:
filter_paraphrases(
    paraphrase_func=paraphrase,
    paraphrase_kwargs=paraphrase_kwargs,
    cls_name='BunnyNoBugs/rubert-tiny2-russe-toxicity',
    max_tries=30
)

{'best_para_text': ('ты сдохнешь в аду за это',
  tensor(0.0024, device='cuda:0', grad_fn=<SelectBackward0>)),
 'para_texts': [('хуйло блядь в рот ебаное пидорас с горы',
   tensor(0.0011, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуйло ебаное быдло чтоб земля под тобой горела',
   tensor(0.0012, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуйло ебаное мразь чтоб ты сдох',
   tensor(0.0012, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуйло ебаное в рот тебе проваливай',
   tensor(0.0011, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуя тебе в пизду а не венера',
   tensor(0.0011, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуйло ебаное пидорас бы тебе в очко',
   tensor(0.0011, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('твоя жопа чтобы на ней вырос хуй',
   tensor(0.0012, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('хуя тебе пиздец ты как ребенок в пизде',
   tensor(0.0012, device='cuda:0', grad_fn=<SelectBackward0>)),
  ('пидорас хуя ты сдохн

In [49]:
para_results = []
for i in tqdm(toxic_inputs):
    para_result = paraphrase(i,
                             model,
                             tokenizer,
                             top_k=20,
                             top_p=1.0
                             )
    para_results.append(para_result)

  0%|          | 3/800 [00:02<08:53,  1.49it/s]


KeyboardInterrupt: 

In [54]:
with open(f'data/output/{model_name}-top-k-20-top-p-1,0_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence + '\n' for sentence in para_results])

In [39]:
model_name

'rugpt3-paraphraser'